In [3]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# other required parameters
from ImageAnalysis3.classes import _allowed_kwds

29044


In [4]:
import pandas as pd

# Read standard adaptor_sequences

In [5]:
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain'
library_folder = os.path.join(pool_folder, r'mouse_genome_1000')
save_folder = os.path.join(pool_folder, 'Summary_tables')


In [9]:

lib_name = '1000-mouse-genome_500'
data_type = 'combo'
adaptor_sequence_filename = os.path.join(save_folder, f'{lib_name}_{data_type}_adaptor_sequences.csv')
print(os.path.exists(adaptor_sequence_filename))

True


In [17]:
adaptor_info = pd.read_csv(adaptor_sequence_filename, header=0)
adaptor_info

,group,hyb,750_bits,750_readouts,647_bits,647_readouts
0,1,1,b0,NDB_784,b1,NDB_755
1,1,2,b2,NDB_826,b3,NDB_713
2,1,3,b4,NDB_865,b5,NDB_725
3,1,4,b6,NDB_817,b7,NDB_710
4,1,5,b8,NDB_652,b9,NDB_971
5,1,6,b10,NDB_718,b11,NDB_743
6,1,7,b12,NDB_847,b13,NDB_683
7,1,8,b14,NDB_643,b15,NDB_815
8,1,9,b16,NDB_760,b17,NDB_965
9,1,10,b18,NDB_790,b19,NDB_917


# load all adaptors

In [14]:
from ImageAnalysis3 import library_tools
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors'
adaptor_files = [os.path.join(adaptor_folder, _fl) for _fl in os.listdir(adaptor_folder) 
                 if _fl.split(os.extsep)[-1]=='fasta' and _fl.split(os.extsep)[-2][-9:] == '_adaptors']
adaptors = library_tools.sequences.fasta_reader(adaptor_files, True).load()

loading 3 fasta files
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors\20200121_extend_stv_adaptors.fasta
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors\NDB_adaptors.fasta
- loading from file: \\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Adaptors\Stv_adaptors.fasta


# add adaptors

In [39]:
adaptor_dict = {}
for _i in adaptor_info.index:
    _adts = []
    cy7_readout_name = adaptor_info.iloc[_i]['750_readouts']
    if isinstance(cy7_readout_name, str):
        cy7_adaptor = [_a for _a in adaptors if cy7_readout_name in _a.id][0]
        _adts.append(cy7_adaptor)
    cy5_readout_name = adaptor_info.iloc[_i]['647_readouts']
    if isinstance(cy5_readout_name, str):
        cy5_adaptor = [_a for _a in adaptors if cy5_readout_name in _a.id][0]
        _adts.append(cy5_adaptor)
    adaptor_dict[_i] = _adts

# Generate df match IDT format

use rows first, skip neighobring rows and columns to fit 24 well plate

In [56]:
len(adaptor_dict)

50

In [65]:
plate_df = pd.DataFrame()

plate_rows = [_c.upper() for _c in 'ABCDEGFH']
plate_cols = np.arange(1,13)

num_rounds = 1 # skip the first round because they are stained on bench

for _row in plate_rows:
    for _col in plate_cols[::2]:
        print(f"Well: {_row}{_col}")
        # loop thorugh adaptor
        for _adt in adaptor_dict[num_rounds]:
            _info_dict = {"Well Position": f"{_row}{_col}"}
            _adt_name = _adt.id
            if _adt_name[-2:] != 'rc':
                _adt_name = _adt_name + 'rc'
            _adt_seq = str(_adt.seq)
            _info_dict['Sequence Name'] = _adt_name
            _info_dict['Sequence'] = _adt_seq
            # append
            plate_df = plate_df.append(_info_dict, ignore_index=True)
        # update
        num_rounds += 1
        if num_rounds not in adaptor_dict:
            break
plate_df = plate_df[['Well Position', 'Sequence Name', 'Sequence']]
plate_df

Well: A1
Well: A3
Well: A5
Well: A7
Well: A9
Well: A11
Well: B1
Well: B3
Well: B5
Well: B7
Well: B9
Well: B11
Well: C1
Well: C3
Well: C5
Well: C7
Well: C9
Well: C11
Well: D1
Well: D3
Well: D5
Well: D7
Well: D9
Well: D11
Well: E1
Well: E3
Well: E5
Well: E7
Well: E9
Well: E11
Well: G1
Well: G3
Well: G5
Well: G7
Well: G9
Well: G11
Well: F1
Well: F3
Well: F5
Well: F7
Well: F9
Well: F11
Well: H1
Well: H3
Well: H5
Well: H7
Well: H9
Well: H11


,Well Position,Sequence Name,Sequence
0,A1,NDB_826_2xStv_82rc,ATATGTCGATGTCCTTAGACTTTGCACTGCCGTCCTTGACTTTGCA...
1,A1,NDB_713_2xStv_1rc,GGTGGTTGTCACTTTCCGAAGATCCGATTGGAACCGTCCCGATCCG...
2,A3,NDB_865_2xStv_82rc,GATCCCGGTCGGATATGTGATTTGCACTGCCGTCCTTGACTTTGCA...
3,A3,NDB_725_2xStv_1rc,CTACCACGGCATCGATTTAGGATCCGATTGGAACCGTCCCGATCCG...
4,A5,NDB_817_2xStv_82rc,GAACTCGTTGACTAAGCATTTTTGCACTGCCGTCCTTGACTTTGCA...
...,...,...,...
91,H7,NDB_818_2xStv_1rc,GGTCGTCTATACCAACCTTAGATCCGATTGGAACCGTCCCGATCCG...
92,H9,NDB_940_2xStv_82rc,TCGGGCTAGCAACTGCGATGTTTGCACTGCCGTCCTTGACTTTGCA...
93,H9,NDB_872_2xStv_1rc,TTAGCGGCTTATTCCTCTACGATCCGATTGGAACCGTCCCGATCCG...
94,H11,NDB_934_2xStv_82rc,TTCGCGCGCATTTGGGAAGTTTTGCACTGCCGTCCTTGACTTTGCA...


In [70]:
# save
plate_filename = os.path.join(save_folder, f'{lib_name}_{data_type}_adaptor_plate.xls')
plate_df.to_excel(plate_filename, header=True, index=False)


E:\Users\puzheng\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
type(cy5_readout_name)

float

TypeError: Cannot index by location index with a non-integer key